# Mecanismo de atenção

Atenção consiste em combinar diferentes informações sendo processadas por uma rede neural de forma a aumentar o contexto dos atributos. Por exemplo, ao processar uma imagem, filtros de CNNs combinam valores bem locais (ex: regiões 3x3), e são necessárias muitas camadas para que a rede consiga combinar regiões distintias de uma imagem. Mecanismos de atenção podem ser utilizados para combinar regiões distantes de uma imagem, o que permite que cada região seja processada com um contexto global.

### Transformação de uma imagem em uma sequência de tokens

A atenção espacial consiste em transformar regiões da imagem em tokens. Esses tokens não possuem uma posição espacial, eles são tratados como um *bag of tokens* que podem ser combinados de forma independente, sem levar em conta a proximidade entre eles. Tokens são exatamente o mesmo conceito encontrado em processamento de língua natural (NLP).

In [1]:
import torch
from torch import nn

class PatchifyLayer(nn.Module):
    """Módulo que transforma uma imagem em um conjunto de tokens."""
        
    def __init__(self, image_size, patch_size, token_dim):
        """`image_size` precisa ser divisível por `patch_size`.

        Args:
            image_size (int): tamanho da imagem que será processada.
            patch_size (int): tamanho das regiões que serão transformada em tokens.
            token_dim (int): número de atributos gerados para cada token.
        """
        super().__init__()

        # Note o stride. Essa camada transforma cada região patch_size x patch_size 
        # da imagem em token_dim x 1 x 1
        self.conv_proj = nn.Conv2d(
            3, token_dim, kernel_size=patch_size, stride=patch_size
        )

        # Novo tamanho da imagem
        new_size = image_size//patch_size
        # Tamanho da sequência de tokens
        seq_length = new_size**2

        self.token_dim = token_dim
        self.new_size = new_size
        self.seq_length = seq_length

    def forward(self, x):

        # (bs, c, image_size, image_size) -> (bs, token_dim, new_size, new_size)
        x = self.conv_proj(x)
        # (bs, token_dim, new_size, new_size) -> (bs, token_dim, (new_size*new_size))
        x = x.reshape(x.shape[0], self.token_dim, -1)
        # Coloca a dimensão espacial como segunda, pois o padrão de camadas de 
        # atenção é bs x seq_length x token_dim
        x = x.permute(0, 2, 1)

        return x

# 8 imagens RGB de tamanho 224 x 224
x = torch.rand(8, 3, 224, 224)
pl = PatchifyLayer(image_size=224, patch_size=16, token_dim=768)
tokens = pl(x)
tokens.shape

torch.Size([8, 196, 768])

Cada imagem em um batch é representada por 196 tokens, cada um possuindo 768 atributos

#### Nota: Multiplicação matricial em batches

No que faremos no restante deste notebook, será importante entender como uma sequência de tokens pode ser processada em batches em uma camada linear

In [7]:
# Dado de entrada
x = torch.rand(196, 768)
# Matrix de pesos
w = torch.rand(768, 64)

# Multiplicação matricial, o resultado possuirá tamanho:
# 196x768 * 768x64 -> 196x64
y = torch.matmul(x, w)
y.shape

torch.Size([196, 64])

O Pytorch possibilita realizar multiplicação matricial em batches, por exemplo:

In [9]:
# Dado de entrada
x = torch.rand(8, 12, 196, 768)
# Matrix de pesos
w = torch.rand(768, 64)

# 8x12x196x768 * 768x64 -> 8x12x196x64
y = torch.matmul(x, w)
y.shape

torch.Size([8, 12, 196, 64])

O código acima é equivalente a fazer algo como:

```python
y = torch.zeros(8, 12, 196, 64)
for idx_dim1 in range(8):
    for idx_dim2 in range(12):
        y[idx_dim1,idx_dim2] = torch.matmul(x[idx_dim1,idx_dim2], w)
```

O mesmo ocorre para camadas lineares:

In [11]:
linear = nn.Linear(768, 64)
y = linear(x)
y.shape

torch.Size([8, 12, 196, 64])

### Operação de atenção

Tendo a imagem representada como uma sequência, podemos aplicar um mecanismo de atenção à imagem

In [2]:
def attention(query, key, value):
    
    # Tamanho de cada token
    d_k = query.shape[-1]
    # Similaridade entre cada par de tokens da sequência
    scores = torch.matmul(query, key.transpose(-2, -1)) / d_k**0.5
    # Normaliza a similaridade entre [0,1]
    p_attn = scores.softmax(dim=-1)
    # Atualiza os valores dos tokens de acordo com as similaridades
    value = torch.matmul(p_attn, value)

    return value

out = attention(tokens, tokens, tokens)
out.shape

torch.Size([8, 196, 768])

Note que a operação de atenção não possui parâmetros treináveis!

### Atenção com múltiplas cabeças

O artigo original sobre atenção define a chamada multi-head attention, que consiste em realizar diversas atenções em paralelo para aumentar o poder de expressividade do modelo.

In [3]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, heads, token_dim):
        super().__init__()

        # Valor usado para normalização
        d_k = token_dim//heads
        self.heads = heads
        self.d_k = d_k
        # Camadas de projeção antes da atenção
        self.proj_query = nn.Linear(token_dim, token_dim)
        self.proj_key = nn.Linear(token_dim, token_dim)
        self.proj_value = nn.Linear(token_dim, token_dim)
        self.final = nn.Linear(token_dim, token_dim)

    def proj_and_reshape(self, layer, x):
        '''Aplica uma transformação linear e redimensiona o resultado
        para ser usado na função de atenção.'''

        bs = x.shape[0]
        # A multiplicação x*layer.weight abaixo possui dimensão:
        # (bs x n x token_dim) * (token_dim x heads*d_k)
        # onde n é o tamanho da sequência.
        # Cada sequência com token_dim (heads*d_k) atributos é multiplicada por 
        # uma coluna da camada linear. Isso é equivalente a fazer a sequinte operação:
        # Aplicar `heads`` camadas, cada uma com tamanho token_dim x d_k, nas sequências
        # e depois concatenar os resultados. 
        x = layer(x)
        # Visualiza o resultado como uma matriz bs x heads x n x d_k. Isso
        # possibilita aplicar a função `attention` nas dimensões n x d_k
        x = x.view(bs, -1, self.heads, self.d_k).transpose(1, 2)

        return x

    def forward(self, query, key, value):

        nbatches = query.shape[0]
        query_proj = self.proj_and_reshape(self.proj_query, query)
        key_proj = self.proj_and_reshape(self.proj_key, key)
        value_proj = self.proj_and_reshape(self.proj_value, value)

        x = attention(query_proj, key_proj, value_proj)
        # Redimensiona de bs x heads x n x d_k para bs x n x token_dim
        x = x.transpose(1, 2).contiguous().view(nbatches, -1, self.heads*self.d_k)

        return self.final(x)
    
mha = MultiHeadedAttention(heads=12, token_dim=768)
out = mha(tokens, tokens, tokens)
out.shape

torch.Size([8, 196, 768])

O Pytorch possui uma camada que faz exatamente o que implementamos:

In [4]:
mha = nn.MultiheadAttention(embed_dim=768, num_heads=12, batch_first=True)
out, attn_weights  = mha(tokens, tokens, tokens)
print(out.shape, attn_weights.shape)

torch.Size([8, 196, 768]) torch.Size([8, 196, 196])


A camada também retorna a multiplicação entre as chaves e queries (variável `p_attn` na nossa função de atenção). Essa variável é útil para verificar o relacionamento entre os tokens da sequência.

In [5]:
print(mha.in_proj_weight.shape)
print(mha.out_proj.weight.shape)

torch.Size([2304, 768])
torch.Size([768, 768])


O atributo `in_proj_weight` é a matriz de pesos das projeções das chaves, queries e valores. O Pytorch realiza as projeções em uma única multiplicação matricial. Para fazer isso basta concatenar as matrizes `proj_query`, `proj_key` e `proj_value` da nossa classe. Não fizemos dessa forma para deixar o código mais simples de entender.

O atributo `out_proj` é a camada linear de saída que implementamos (`final`)

Até o momento utilizamos a chamada *self-attention*, que consiste em utilizar a mesma variável como query, key e value. Mas o mecanismo de atenção pode ser utilizado de forma natural para misturar diferentes informações. Por exemplo, uma camada de atenção pode receber como entrada atributos sobre um texto e sobre uma imagem. Nesse caso, é comum associar `key` e `value` com os atributos do texto e `query` com os atributos da imagem:

In [6]:
# batch com 8 sequências de tokens de imagens
tokens_img = torch.rand(8, 196, 768)
# batch com 8 sequências de tokens de texto, cada texto possui 20 tokens e
# cada token 512 atributos. Por exemplo, esses textos podem ser descrições
# da imagem como "Uma imagem de um cachorro dormindo"
tokens_text = torch.rand(8, 20, 768)

#              query         key         value
out, _ = mha(tokens_img, tokens_text, tokens_text)
out.shape

torch.Size([8, 196, 768])

Os nomes *key*, *value* e *query* referenciam conceitos de banco de dados. Podemos considerar que *key* e *value* representam as chaves e valores de itens de um banco de dados. *query* é uma busca feita no banco. A similaridade entre a busca (*query*) e as chaves (*keys*) é calculada. A busca vai ser mais similar a algumas chaves do que outras. Os elementos mais similares encontrados na busca são usados para atualizar os valores de *value*, e esses valores atualizados são a saída da camada. 